In [177]:
import os
import pandas as pd
import uuid
from datetime import datetime
import mysql.connector as msql
from mysql.connector import Error

In [178]:
sample = "full"

In [1]:
def check_labels_file(folder):
    subfolders = os.listdir(folder)
    dfs = []
    for i, sf in enumerate(subfolders):
        user_folder = os.path.join(folder,sf)
        labels_file = os.path.join(user_folder, 'labels.txt')
        if os.path.exists(labels_file):
            print('Reading user %s ---------------------------' % (sf))
# check_labels_file('Datasets/'+sample+'_sample')

In [180]:
''' 
Ao ler os usuários, verificar a existência do arquivo labels.txt
'''

mydb = msql.connect(
  host="localhost",
  user="root",
  password="root",
  database="trajectories"
)
mycursor = mydb.cursor()

def processing_trajectories(folder):
    subfolders = os.listdir(folder)
    dfs = []
    
    for i, sf in enumerate(subfolders):
        user_folder = os.path.join(folder,sf)
        labels_file = os.path.join(user_folder, 'labels.txt')
        
        print('Reading user %s ---------------------------' % (sf))
        
        u_transport = pd.read_csv(labels_file, sep="\t")
        u_transport['Identifier'] = [str(uuid.uuid1()) for x in range(u_transport.shape[0])]
        insert_trajectories_into_db(u_transport)
            
        plt_files = os.path.join(user_folder, 'Trajectory')
        for filename in os.listdir(plt_files): 
            u_gps_points = pd.read_csv(os.path.join(plt_files, filename), skiprows=6, header=None,
                                  parse_dates=[[5, 6]], infer_datetime_format=True)
            u_gps_points.rename(inplace=True, columns={'5_6': 'Timestamp', 0: 'Latitude', 
                                                  1: 'Longitude', 3: 'Altitude'})
            u_gps_points.drop(inplace=True, columns=[2, 4])
            
            u_gps_points_labeled = select_labeled_points(u_transport,u_gps_points)

            insert_gpspoints_into_db(u_gps_points_labeled)
            
def select_labeled_points(transport,gps_points):
    gps_points_labeled = pd.DataFrame(columns=['Timestamp', 'Latitude', 'Longitude', 'Altitude', 'Label', 'Identifier'])
    count = 1
    total = gps_points.shape[0]
    print('Selecting and labeling gps points of trajectories')
    for index_transport,row_transport in transport.iterrows():
        print('----------------------------------------------------> (%s/%s)' % (count,total))
        
        identifier = row_transport['Identifier']
        start = datetime.strptime(row_transport['Start Time'],'%Y/%m/%d %H:%M:%S')
        end = datetime.strptime(row_transport['End Time'],'%Y/%m/%d %H:%M:%S')
        label = row_transport['Transportation Mode']
        
        for index_gps,row_gps in gps_points.iterrows():
            if start <= row_gps['Timestamp'] <= end:
                
                gps_points_labeled.loc[len(gps_points_labeled.index)] = [row_gps['Timestamp'], row_gps['Latitude'],
                                                                         row_gps['Longitude'], row_gps['Altitude'],  
                                                                         label, identifier]
            
        count += 1
                
    return gps_points_labeled
    
def insert_gpspoints_into_db(gps_points):
    count = 1
    total = gps_points.shape[0]
    print('Inserting gps points')
    for index_gpspoints,row_gpspoints in gps_points.iterrows():
        print('-------------------------> (%s/%s)' % (count,total))
                        
        try:
            if mydb.is_connected():

                sql = """INSERT INTO trajectories.gpspoints (timestamp, latitude, longitude, altitude, 
                transport_mode, identifier) VALUES (%s, %s, %s, %s, %s, %s)"""
                val = tuple(row_gpspoints.values)
                mycursor.execute(sql, val)

                mydb.commit()

                print("--> record inserted.")

                count += 1
        
        except Error as e:
            
            print("Error while connecting to gpspoints:", e)

def insert_trajectories_into_db(transport_df):
    count = 1
    total = transport_df.shape[0]
    print('Inserting trajectories')
    for index_transport,row_transport in transport_df.iterrows():
        print('-------------------------> (%s/%s)' % (count,total))
                        
        try:
            if mydb.is_connected():
                
                sql = """INSERT INTO trajectories.trajectories (start, end, transport_mode, identifier) 
                VALUES (%s, %s, %s, %s)"""
                val = tuple(row_transport.values)
                mycursor.execute(sql, val)

                mydb.commit()

                print("--> record inserted.")

                count += 1
        
        except Error as e:
            
            print("Error while connecting to trajectories_table:", e)    

In [2]:
# processing_trajectories('Datasets/'+sample+'_sample')